In [1]:
import numpy as np
from pyfeti.src.feti_solver import SerialFETIsolver
from scipy.sparse import linalg

K1 = K3 = np.array([[1, 0, 0, 0],
                    [0, 1, -1, 0],
                    [0, 0, 1, -1],
                    [0, 0, 0, 1]])

K2 = K4 = np.array([[1, -1, 0, 0],
                    [0, 1, 0, 0],
                    [0, 0, 1, 0],
                    [-1, 0, 0, 1]])

q0 = 1.0
q1 = np.array([0.,0.,0.,0.])
q2 = np.array([0.,0.,0.,0.])
q3 = np.array([0.,0.,0.5,0.])*q0
q4 = np.array([0.,0.,0.,0.5])*q0

B12 =  np.array([[0,1,0,0],
                 [0,0,1,0]])

B13 = np.array([[0,0,1,0],
                [0,0,0,1]])

B21 =  np.array([[-1,0,0,0],
                 [0,0,0,-1]])

B24 = np.array([[0,0,1,0],
                [0,0,0,1]])

B31 = np.array([[0,-1,0,0],
                [-1,0,0,0]])

B34 = np.array([[0,1,0,0],
                [0,0,1,0]])

B42 = np.array([[0,-1,0,0],
                [-1,0,0,0]])

B43 = np.array([[-1,0,0,0],
                [ 0,0,0,-1]])


# Using PyFETI to solve the probrem described above
K_dict = {1:K1,2:K2, 3:K3, 4:K4}
B_dict = {1 : {(1,2) : B12, (1,3) : B13}, 
          2 : {(2,1) : B21, (2,4) : B24}, 
          3 : {(3,1) : B31, (3,4) : B34}, 
          4 : {(4,2) : B42, (4,3) : B43}}

q_dict = {1:q1 ,2:q2, 3:q3, 4:q4}


In [2]:
solver_obj = SerialFETIsolver(K_dict,B_dict,q_dict)

solution_obj = solver_obj.solve()

T_dual = solution_obj.displacement
lambda_ = solution_obj.interface_lambda
alpha =  solution_obj.alpha

In [3]:
T_dual

array([0.        , 0.03114578, 0.08468567, 0.02331592, 0.03114578,
       0.02567112, 0.04293025, 0.08468567, 0.02331592, 0.08468567,
       0.45657332, 0.26176559, 0.08468567, 0.04293025, 0.        ,
       0.45657332])

In [4]:
lambda_

array([-0.00654289,  0.00454132, -0.08830518,  0.02331592, -0.04293025,
       -0.09558532,  0.00090125,  0.08111535])

In [5]:
alpha

array([], dtype=float64)

In [6]:
B = solver_obj.manager.assemble_global_B()
K, f = solver_obj.manager.assemble_global_K_and_f()
G = solver_obj.manager.assemble_G()

In [7]:
B.dot(T_dual)

array([ 1.06428234e-11,  1.08541370e-11,  2.02154959e-12,  4.40979545e-12,
       -1.16320495e-11, -7.50913220e-13,  8.08167422e-12,  1.50340851e-11])

In [8]:
K_inv = np.linalg.inv(K.A)
F = B@K_inv@B.T
d = B@K_inv@f

In [10]:
lambda_cg, info = linalg.cg(F,d)

In [11]:
lambda_cg

array([ 0.08333333,  0.04166667, -0.20833333,  0.        ,  0.        ,
       -0.04166667,  0.20833333, -0.08333333])

In [12]:
lambda_


array([-0.00654289,  0.00454132, -0.08830518,  0.02331592, -0.04293025,
       -0.09558532,  0.00090125,  0.08111535])

In [13]:
T_cg = K_inv@(f - B.T@lambda_cg)

In [14]:
T_feti = K_inv@(f - B.T@lambda_)

In [15]:
T_cg

array([0.        , 0.08333333, 0.16666667, 0.        , 0.08333333,
       0.        , 0.        , 0.16666667, 0.        , 0.16666667,
       0.58333333, 0.        , 0.16666667, 0.        , 0.        ,
       0.58333333])

In [16]:
T_feti

array([ 0.        ,  0.06699082,  0.06044793, -0.02331592, -0.00654289,
        0.        ,  0.04293025,  0.09358376,  0.02331592,  0.32967823,
        0.41888465,  0.        , -0.13761433, -0.04293025,  0.        ,
        0.44350102])

In [18]:
eigval, eigvec = np.linalg.eig(F)

eigval

array([ 4.73205081e+00+0.00000000e+00j,  1.26794919e+00+0.00000000e+00j,
        2.00000644e+00+1.11516508e-05j,  2.00000644e+00-1.11516508e-05j,
        1.99998712e+00+0.00000000e+00j, -4.86636234e-17+0.00000000e+00j,
        2.00000000e+00+0.00000000e+00j,  2.00000000e+00+0.00000000e+00j])

In [19]:
F

array([[ 2.,  1.,  1.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  2.,  1.,  1.,  0., -1.,  0.,  0.],
       [ 0.,  1.,  2.,  1.,  0.,  0., -1., -1.],
       [ 0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.],
       [-1., -1.,  0.,  0.,  1.,  2.,  1.,  0.],
       [ 0.,  0., -1.,  0.,  1.,  1.,  2.,  1.],
       [ 0.,  0.,  0.,  0.,  1.,  1.,  1.,  2.]])